# Transversers

This notebook describes part of [Emma](https://github.com/emmalanguage/emma)'s compiler API. Transversers (short for transformers + traversers) are focused on manipulating Scala trees and Emma IR trees. Transformers change the tree while traversers only collect information about it. For all intents and purposes, both are equivalent (we can see traversal as the identity transformation). The only practical difference is efficiency.

The gist if this API is avoiding explicit recursion and mutable state by defining rules that operate on one node (and optionally some attributes) at a time. Each rule is usually one `case` in a partial function (also referred to as "Matcher") that returns substitute nodes or attributes and is passed to second-order combinators that specify the overall recursion strategy.

## Setup
This section explains step by step how to setup the notebook. We assume that you have followed the instructions from [README](./README.md) and are running this notebook from the `emma-jupyter` directory.
- We need to load the `emma-language` artifact to gain access to Emma's APIs:

In [ ]:
import java.nio.file.Paths

// Check the current directory (should be emma-jupyter):
println(sys.props("user.dir"))

// Register the local Maven repository (substitute HOME if necessary):
val HOME = sys.props("user.home")
classpath.addRepository(s"file://$HOME/.m2/repository/")

// Add emma-language to the classpath:
classpath.add("eu.stratosphere" % "emma-language" % "1.0-SNAPSHOT")

// Add the test-classes from emma-language to the classpath:
val testClasses = Paths.get("../emma-language/target/test-classes").toAbsolutePath().normalize().toString()
classpath.addPath(testClasses)

- Next, we create a runtime compiler to test out features of the API:

In [3]:
import eu.stratosphere.emma.compiler._
import cats.std.all._

val compiler = new RuntimeCompiler()

// Import APIs that we want to use.
val api = compiler.api
val uni = compiler.universe
val src = compiler.Source.Lang
val core = compiler.Core.Lang

import eu.stratosphere.emma.compiler._
import cats.std.all._
compiler: RuntimeCompiler = eu.stratosphere.emma.compiler.RuntimeCompiler@63295fd4
api: $user.compiler.api.type = eu.stratosphere.emma.ast.AST$api$@577b8509
uni: reflect.runtime.JavaUniverse = scala.reflect.runtime.JavaUniverse@676036c0
src: $user.compiler.Source.Lang.type = eu.stratosphere.emma.compiler.lang.source.Source$Source$Lang$@590325f2
core: $user.compiler.Core.Lang.type = eu.stratosphere.emma.compiler.lang.core.Core$Core$Lang$@2cdd3fc4

For more details on `Core.Lang` see the [Core language notebook](./CoreLanguage.ipynb) and for details on `Source.Lang` see the [Source language notebook](./SourceLanguage.ipynb).

## Dissecting a Transformation / Traversal

Transformations and traversals consist of four major components:

1. Strategy - the recursion scheme;
2. Attributes - additional node information for the rules;
3. Rules - the matcher function;
4. Result - any post-processing if necessary.

Next, we're going to look at each one in detail.

### Strategies

The transform / traverse API features six fundamental recursion strategies characterized in two orthogonal dimensions:

1. Vertical direction:
    - **Top-down**: traversal / transformation descend in a depth-first fashion from the root to the leaves;
    - **Bottom-up**: traversal / transformation ascend recursively from the leaves to the root.
2. After-match behaviour (what happens after a match at the current node):
    - **Continue**: traversal / transformation continue to the next node in order;
    - **Break**: traversal / transformation stop descending or ascending (NOTE: to stop ascending all children have to match);
    - **Exhaust**: traversal / transformation iterate at the current node until a fix-point is found, then continue.

The combination of the above determines the direction and extent to which rules are applied. Theoretically, we can differentiate a third dimension - horizontal direction (left-to-right vs. right-to-left), but in practice we adopt a left-to-right policy.

The table below illustrates the differences between strategies based on a simplified AST example. Green nodes are potential rule matches, whereas red nodes are ignored. The traversal / transformation path is shown as blue arrows.

| Top-down Strategies | Bottom-up Strategies |
|:-------------------:|:--------------------:|
| *Top-down Continue* ![](img/top-down-continue.png) `api.TopDown` | *Bottom-up Continue* ![](img/bottom-up-continue.png) `api.BottomUp` |
| *Top-down Break* ![](img/top-down-break.png) `api.TopDown.break` | *Bottom-up Break* ![](img/bottom-up-break.png) `api.BottomUp.break` |
| *Top-down Exhaust* ![](img/top-down-exhaust.png) `api.TopDown.exhaust` | *Bottom-up Exhaust* ![](img/bottom-up-exhaust.png) `api.BottomUp.exhaust` |

### 2. Attributes

While traversing or transforming ASTs, it's often necessary to annotate nodes with attributes that provide additional information for implementing rules. The attributes themselves are also generated with rules. The type of attributes is user-defined, with two important restrictions:

1. Attributes are [Monoids](http://typelevel.org/cats/tut/monoid.html) - the identity element is used for nodes that don't match and the binary operation combines previously generated attributes.
2. Attributes are [HLists](https://github.com/milessabin/shapeless/wiki/Feature-overview:-shapeless-2.0.0#heterogenous-lists) (heterogenous lists). You can think of `HList`s as generalized tuples. NOTE: new attributes are prepended which means they appear in reverse order when matching them in rules.

There are three flavors of attributes supported:

**1. Accumulated attributes** - generated along the traversal / transformation path and therefore their semantics depends on the strategy (see previous section for details). You can think of these attributes as a single accumulator that is updated at every visited node according to the associated Monoid. Example below:

In [5]:
// Keep track of all method parameters seen so far in a Vector.
// The default monoid for Vectors and other sequences is concatenation.
api.TopDown.accumulate { case core.DefDef(_, _, _, paramss, _) =>
  (for (core.ParDef(lhs, _, _) <- paramss.flatten) yield lhs).toVector
}

res4: compiler.Strategy[shapeless.::[Vector[compiler.u.TermSymbol], shapeless.HNil], shapeless.HNil, shapeless.HNil] = Strategy(AttrGrammar(<function1>,<function1>,<function1>,false),eu.stratosphere.emma.ast.Transversers$TransFactory$topDown$@15a06cec)

An accumulate attribute can use other accumulated, inherited or synthesized

**2. Inherited attributes** - generated along the path from the root to the current node. You can think of these attributes as a stack that is pushed when entering a subtree and popped when leaving it. For illustration see the next image a few cells down. Example below:

| Synthesized Attributes | Inherited Attributes |
|:----------------------:|:--------------------:|
|![](img/attr-syn.png)   |![](img/attr-inh.png) |